In [3]:
import pandas as pd

# Load the Excel file
file_path = r"C:\Users\sonur\OneDrive\Desktop\Car_Dheko\Raw_Data\bangalore_cars.xlsx"
df = pd.read_excel(file_path)
print(df.head())


                                      new_car_detail  \
0  {'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...   
1  {'it': 0, 'ft': 'Petrol', 'bt': 'SUV', 'km': '...   
2  {'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...   
3  {'it': 0, 'ft': 'Petrol', 'bt': 'Sedan', 'km':...   
4  {'it': 0, 'ft': 'Diesel', 'bt': 'SUV', 'km': '...   

                                    new_car_overview  \
0  {'heading': 'Car overview', 'top': [{'key': 'R...   
1  {'heading': 'Car overview', 'top': [{'key': 'R...   
2  {'heading': 'Car overview', 'top': [{'key': 'R...   
3  {'heading': 'Car overview', 'top': [{'key': 'R...   
4  {'heading': 'Car overview', 'top': [{'key': 'R...   

                                     new_car_feature  \
0  {'heading': 'Features', 'top': [{'value': 'Pow...   
1  {'heading': 'Features', 'top': [{'value': 'Pow...   
2  {'heading': 'Features', 'top': [{'value': 'Pow...   
3  {'heading': 'Features', 'top': [{'value': 'Pow...   
4  {'heading': 'Features', 'top': [{'value': '

In [3]:
import pandas as pd
import ast
import os

# Define file paths
file_path = r"C:\Users\sonur\OneDrive\Desktop\Car_Dheko\Raw_Data\bangalore_cars.xlsx"
output_file_path = r"C:\Users\sonur\OneDrive\Desktop\Car_Dheko\Processed_Data\Bangalore_structured.xlsx"

# Read the Excel file
df = pd.read_excel(file_path)

# Convert JSON strings to dictionaries in 'new_car_detail'
df['new_car_detail'] = df['new_car_detail'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Extract specific details from 'new_car_detail'
df['fuel_type'] = df['new_car_detail'].apply(lambda x: x.get('ft'))
df['body_type'] = df['new_car_detail'].apply(lambda x: x.get('bt'))
df['kilometers_driven'] = df['new_car_detail'].apply(lambda x: x.get('km'))
df['transmission'] = df['new_car_detail'].apply(lambda x: x.get('transmission'))
df['owner'] = df['new_car_detail'].apply(lambda x: x.get('owner'))
df['oem'] = df['new_car_detail'].apply(lambda x: x.get('oem'))
df['model'] = df['new_car_detail'].apply(lambda x: x.get('model'))
df['year'] = df['new_car_detail'].apply(lambda x: x.get('modelYear'))
df['variant'] = df['new_car_detail'].apply(lambda x: x.get('variantName'))
df['price'] = df['new_car_detail'].apply(lambda x: x.get('price'))

# Convert JSON strings to dictionaries in 'new_car_overview'
df['new_car_overview'] = df['new_car_overview'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Extract specific details from 'new_car_overview'
def extract_overview_details(data):
    details = {}
    for item in data.get('top', []):
        details[item['key']] = item['value']
    return details

# Apply the extraction function
overview_details = df['new_car_overview'].apply(extract_overview_details).apply(pd.Series)

# Assign extracted details to new columns
df['registration_year'] = overview_details.get('Registration Year')
df['insurance_validity'] = overview_details.get('Insurance Validity')
df['fuel_type_overview'] = overview_details.get('Fuel Type')
df['seats'] = overview_details.get('Seats')
df['kms_driven'] = overview_details.get('Kms Driven')
df['rto'] = overview_details.get('RTO')

# Convert JSON strings to dictionaries in 'new_car_feature'
df['new_car_feature'] = df['new_car_feature'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Function to extract features based on heading and subHeading
def extract_features(data, heading, subheading=None):
    for item in data['data']:
        if item['heading'] == heading and (subheading is None or item['subHeading'] == subheading):
            return ', '.join([feature['value'] for feature in item['list']])
    return None

# Extract specific features under different headings and subheadings
df['comfort_convenience'] = df['new_car_feature'].apply(lambda x: extract_features(x, 'Comfort & Convenience', 'Comfort'))
df['interior_features'] = df['new_car_feature'].apply(lambda x: extract_features(x, 'Interior', 'Interior'))
df['exterior_features'] = df['new_car_feature'].apply(lambda x: extract_features(x, 'Exterior', 'Exterior'))
df['safety_features'] = df['new_car_feature'].apply(lambda x: extract_features(x, 'Safety', 'Safety'))
df['entertainment_features'] = df['new_car_feature'].apply(lambda x: extract_features(x, 'Entertainment & Communication', 'Entertainment'))

# Convert JSON strings to dictionaries in 'new_car_specs'
df['new_car_specs'] = df['new_car_specs'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Function to extract specifications based on heading and subHeading
def extract_specifications(data, heading, subheading=None):
    for item in data['data']:
        if item['heading'] == heading and (subheading is None or item['subHeading'] == subheading):
            print(f"Found {heading} under {subheading}: {item['list']}")  # Debugging
            return {spec['key']: spec['value'] for spec in item['list']}
    return {}

# Apply the function and inspect the structure
df['specifications'] = df['new_car_specs'].apply(lambda x: extract_specifications(x, 'Engine and Transmission', 'Engine'))

# Display the specifications to check if 'Mileage' exists
print(df['specifications'].head())  # This will display the extracted specifications in the console

# Check for 'Mileage' in the extracted specifications
df['mileage'] = df['specifications'].apply(lambda x: x.get('Mileage'))

# Further processing (like 'engine', 'max_power', etc.) can go here...

# Ensure the output directory exists
os.makedirs(os.path.dirname(output_file_path), exist_ok=True)

# Save the structured data to an Excel file
df.to_excel(output_file_path, index=False)

print("Data processing complete! Structured data saved to:", output_file_path)


Found Engine and Transmission under Engine: [{'key': 'Color', 'value': 'White'}, {'key': 'Engine Type', 'value': 'K10B Engine'}, {'key': 'Displacement', 'value': '998'}, {'key': 'Max Power', 'value': '67.04bhp@6000rpm'}, {'key': 'Max Torque', 'value': '90Nm@3500rpm'}, {'key': 'No of Cylinder', 'value': 3}, {'key': 'Values per Cylinder', 'value': 4}, {'key': 'Value Configuration', 'value': 'DOHC'}, {'key': 'Fuel Suppy System', 'value': 'MPFi'}, {'key': 'BoreX Stroke', 'value': '73 X 82 mm'}, {'key': 'Compression Ratio', 'value': '11.0:1'}, {'key': 'Turbo Charger', 'value': 'No'}, {'key': 'Super Charger', 'value': 'No'}]
Found Engine and Transmission under Engine: [{'key': 'Color', 'value': 'White'}, {'key': 'Engine Type', 'value': 'Ti-VCT Petrol Engine'}, {'key': 'Displacement', 'value': '1497'}, {'key': 'Max Power', 'value': '121.31bhp@6500rpm'}, {'key': 'Max Torque', 'value': '150Nm@4500rpm'}, {'key': 'No of Cylinder', 'value': 3}, {'key': 'Values per Cylinder', 'value': 4}, {'key': '

In [5]:
import pandas as pd
import ast
import os

# Define file paths
file_path = r"C:\Users\sonur\OneDrive\Desktop\Car_Dheko\Raw_Data\bangalore_cars.xlsx"
output_file_path = r"C:\Users\sonur\OneDrive\Desktop\Car_Dheko\Processed_Data\Bangalore_Mil.xlsx"

# Read the Excel file
df = pd.read_excel(file_path)

# Convert JSON strings to dictionaries in 'new_car_specs'
df['new_car_specs'] = df['new_car_specs'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Function to inspect the entire structure of 'new_car_specs'
def inspect_specs(data):
    for item in data['data']:
        print("Heading:", item.get('heading'))
        print("Subheading:", item.get('subHeading'))
        for spec in item.get('list', []):
            print(f"Key: {spec.get('key')}, Value: {spec.get('value')}")
        print("="*50)  # Divider between sections

# Apply the inspection function to the first few rows
df['new_car_specs'].head(5).apply(inspect_specs)

# Function to find and extract mileage from 'new_car_specs'
def find_mileage(data):
    for item in data['data']:
        for sub_item in item['list']:
            if 'mileage' in sub_item['key'].lower():
                return sub_item['value']
    return None

# Apply the function to find mileage
df['mileage'] = df['new_car_specs'].apply(find_mileage)

# Print the first few rows to see if mileage is extracted
print(df[['mileage', 'new_car_specs']].head(10))  # Check the results

# Function to extract specifications based on heading and subheading
def extract_specifications(data, heading, subheading=None):
    for item in data['data']:
        if item['heading'] == heading and (subheading is None or item['subHeading'] == subheading):
            return {spec['key']: spec['value'] for spec in item['list']}
    return {}

# Extract specifications under different headings and subheadings
specifications = df['new_car_specs'].apply(lambda x: extract_specifications(x, 'Engine and Transmission', 'Engine'))
specifications_df = pd.json_normalize(specifications)

# Assign extracted details to new columns with error handling
df['engine'] = specifications_df.get('Engine', None)
df['max_power'] = specifications_df.get('Max Power', None)
df['torque'] = specifications_df.get('Torque', None)
df['wheel_size'] = specifications_df.get('Wheel Size', None)

# Separate max power into bhp and rpm, handling potential missing values
df[['bhp', 'rpm']] = df['max_power'].str.extract(r'(?P<bhp>\d+\.\d+)bhp@(?P<rpm>\d+)rpm')

# Consolidate all extracted data into a new column
df['consolidated_data'] = df.apply(lambda row: {
    'mileage': row['mileage'],
    'engine': row['engine'],
    'max_power': row['max_power'],
    'torque': row['torque'],
    'wheel_size': row['wheel_size'],
    'bhp': row['bhp'],
    'rpm': row['rpm'],
}, axis=1)

# Drop the original columns
df = df.drop(columns=['new_car_specs'])

# Ensure the output directory exists
os.makedirs(os.path.dirname(output_file_path), exist_ok=True)

# Save the structured data to an Excel file
df.to_excel(output_file_path, index=False)

print("Data processing complete! Structured data saved to:", output_file_path)


Heading: Engine and Transmission
Subheading: Engine
Key: Color, Value: White
Key: Engine Type, Value: K10B Engine
Key: Displacement, Value: 998
Key: Max Power, Value: 67.04bhp@6000rpm
Key: Max Torque, Value: 90Nm@3500rpm
Key: No of Cylinder, Value: 3
Key: Values per Cylinder, Value: 4
Key: Value Configuration, Value: DOHC
Key: Fuel Suppy System, Value: MPFi
Key: BoreX Stroke, Value: 73 X 82 mm
Key: Compression Ratio, Value: 11.0:1
Key: Turbo Charger, Value: No
Key: Super Charger, Value: No
Heading: Dimensions & Capacity
Subheading: Dimensions
Key: Length, Value: 3715mm
Key: Width, Value: 1635mm
Key: Height, Value: 1565mm
Key: Wheel Base, Value: 2425mm
Key: Front Tread, Value: 1420mm
Key: Rear Tread, Value: 1410mm
Key: Kerb Weight, Value: 835kg
Key: Gross Weight, Value: 1250kg
Heading: Miscellaneous
Subheading: Miscellaneous
Key: Gear Box, Value: 5 Speed 
Key: Drive Type, Value: FWD
Key: Seating Capacity, Value: 5
Key: Steering Type, Value: Power
Key: Turning Radius, Value: 4.7 metres
K

In [20]:

pip install pandas


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

# Load your DataFrame (make sure to adjust the path as needed)
df = pd.read_excel(r"C:\Users\sonur\OneDrive\Desktop\Car_Dheko\Processed_Data\Bangalore_structured.xlsx")

# Preprocessing
df['kilometers_driven'] = df['kilometers_driven'].str.replace(',', '').astype(float)  # Remove commas and convert to float
df['owner'] = df['owner'].str.replace(' Owner', '')  # Remove the word "Owner"
df['registration_year'] = df['registration_year'].str.extract(r'(\d+)')[0]  # Keep only numeric year
df['seats'] = df['seats'].str.replace(' Seats', '')  # Remove the word "Seats"

# Fill NaN values with 'N/A' only in object-type columns
for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].fillna('N/A')  # Reassign filled values back to the DataFrame

# Save the preprocessed data to CSV
output_file_path_csv = r"C:\Users\sonur\OneDrive\Desktop\Car_Dheko\Processed_Data\Bangalore_preprocessed.csv"
df.to_csv(output_file_path_csv, index=False)

# Display the preprocessed DataFrame
print(df)


           City                                          car_links fuel_type  \
0     Bangalore  https://www.cardekho.com/used-car-details/used...    Petrol   
1     Bangalore  https://www.cardekho.com/buy-used-car-details/...    Petrol   
2     Bangalore  https://www.cardekho.com/used-car-details/used...    Petrol   
3     Bangalore  https://www.cardekho.com/buy-used-car-details/...    Petrol   
4     Bangalore  https://www.cardekho.com/used-car-details/used...    Diesel   
...         ...                                                ...       ...   
1476  Bangalore  https://www.cardekho.com/used-car-details/used...    Diesel   
1477  Bangalore  https://www.cardekho.com/used-car-details/used...    Petrol   
1478  Bangalore  https://www.cardekho.com/used-car-details/used...    Petrol   
1479  Bangalore  https://www.cardekho.com/used-car-details/used...    Diesel   
1480  Bangalore  https://www.cardekho.com/used-car-details/used...    Petrol   

      body_type  kilometers_driven tran